# Compiled Code Logic Simulator

Capable of simulating combinational logic represented using AND, NOT and OR gates

### We shall use `.txt` netlist file (SPICE like style), to enter the graph configuration

#### An example input code for this circuit

```spice
inp n1 n2 n4 n6
outp n3 n5 n7

a n1 n2 n3
o n3 n4 n5
i n6 n7
```

> ```inp nodes``` : denotes inputs to circuit

> ```outp nodes```  : denotes outputs to circuit

> ```a n1 n2 n3```  : denotes and gate with inputs n1 and n2, output as n3

> ```o n3 n4 n5```  : denotes or gate with inputs n3 and n4, output as n5 

> ```i n6 n7```  : denotes inverter with input n6 and output n7
 

In [1]:
class notgate:
    def __init__(self, a, y):
        self.a = a
        self.y = y
    
    def __repr__(self):
        return "NOT Gate"
    
    def ready(self, nets):
        return (nets[self.a] != 'u') 
    
    def evaluate(self, nets):
        if nets[self.a] == 1:
            nets[self.y] = 0
        else:
            nets[self.y] = 1

class andgate:
    def __init__(self, a, b, y):
        self.a = a
        self.b = b
        self.y = y
    
    def __repr__(self):
        return "AND Gate"

    def ready(self, nets):
        return (nets[self.a] != 'u' and nets[self.b] != 'u') 

    def evaluate(self, nets):
        if(nets[self.a] and nets[self.b]):
            nets[self.y] = 1
        else:
            nets[self.y] = 0

class orgate:
    def __init__(self, a, b, y):
        self.a = a
        self.b = b
        self.y = y   
    
    def __repr__(self):
        return "OR Gate"
    
    def ready(self, nets):
        return (nets[self.a] != 'u' and nets[self.b] != 'u') 
 
    def evaluate(self, nets):
        if(nets[self.a] or nets[self.b]):
            nets[self.y] = 1
        else:
            nets[self.y] = 0

In [2]:
def get_input_output_nodes(filename):
    inputs = []
    outputs = []
    
    with open(filename, 'r') as t:
        gate_number = 0
        m = [x.split() for x in t.readlines()]
        for i in m:
            if(len(i) == 0):
                pass
            elif(i[0] == 'inp'):
                inputs = i[1:]
            elif(i[0] == 'outp'):
                outputs = i[1:]
            else:
                pass
    return (inputs, outputs)

def simulate(filename, testvector):
    inputs = []
    outputs = []

    nets = {}
    gates = {}

    with open(filename, 'r') as t:
        gate_number = 0
        m = [x.split() for x in t.readlines()]
        for i in m:
            if(len(i) == 0):
                pass
            elif(i[0] == 'inp'):
                inputs = i[1:]
            elif(i[0] == 'outp'):
                outputs = i[1:]
            elif(i[0] == 'a'):
                for j in range(1,4):
                    if(i[j] not in nets.keys()):
                        nets[i[j]] = 'u'
                gates[gate_number] = andgate(i[1], i[2], i[3])
                gate_number += 1
            elif(i[0] == 'o'):
                for j in range(1,4):
                    if(i[j] not in nets.keys()):
                        nets[i[j]] = 'u'
                gates[gate_number] = orgate(i[1], i[2], i[3])
                gate_number += 1
            elif(i[0] == 'i'):
                for j in range(1,3):
                    if(i[j] not in nets.keys()):
                        nets[i[j]] = 'u'
                gates[gate_number] = notgate(i[1], i[2])
                gate_number += 1
            else:
                pass
            
    for i in range(len(testvector)):
        nets[inputs[i]] = testvector[i]
    
    left_over_gates = list(range(len(gates.keys())))
    while(len(left_over_gates) > 0):
        completed = []
        
        for i in left_over_gates:
            if(gates[i].ready(nets)):
                completed.append(i)
        
        for i in completed:
            gates[i].evaluate(nets)
            left_over_gates.remove(i)
    
    output_testvector = []
    
    for i in outputs:
        output_testvector.append(nets[i])
    
    return output_testvector

## Print Truth Table for Logic Function

In [3]:
from itertools import product

inputs, outputs = get_input_output_nodes('circuit.txt')

truthtable = {}

for j in range(len(inputs)):
    truthtable[inputs[j]] = []

for j in range(len(outputs)):
    truthtable[outputs[j]] = []
    
testvectors = list(product((0, 1), repeat=len(inputs)))

for i in testvectors:
    for j in range(len(i)):
        truthtable[inputs[j]].append(i[j])
    
    v = simulate('circuit.txt', i)

    for j in range(len(v)):
        truthtable[outputs[j]].append(v[j])

In [4]:
import pandas as pd
from IPython.display import HTML

truthtable = pd.DataFrame(truthtable)
truthtable.to_csv("truthtable.csv", index=False)

print("The truth table for the circuit is given by")
HTML(truthtable.to_html(index=False))

The truth table for the circuit is given by


s,p,q,y
0,0,0,0
0,0,1,0
0,1,0,1
0,1,1,1
1,0,0,0
1,0,1,1
1,1,0,0
1,1,1,1
